In [45]:
import pandas as pd
import pandasql as pdsql

In [46]:
employer = pd.read_csv("company_name.csv")

In [47]:
employer

,member_id,company_name,year_start
0,1,Microsoft,2007
1,1,Linkedin,2009
2,1,Microsoft,2010
3,1,Google,2013
4,2,Microsoft,2005
5,2,Google,2009
6,2,Apple,2011
7,3,LinkedIn,2001
8,3,Google,2002
9,3,Microsoft,2004


In [26]:
# (1) count members who ever moved from Microsoft to Google?
min_ms = employer[employer.company_name == "Microsoft"].groupby("member_id")["year_start"].min().to_frame()
max_gg = employer[employer.company_name == "Google"].groupby("member_id")["year_start"].max().to_frame()
match = min_ms.merge(max_gg, left_index = True, right_index = True, how = "inner")
match[match.year_start_x <= match.year_start_y].shape[0]

3

In [28]:
#(2) count members who directly moved from Microsoft to Google? (Microsoft -- Linkedin -- Google doesn't count)

In [51]:
employer["rank"] = employer.sort_values(["member_id","year_start","company_name"]).\
                            groupby("member_id").cumcount() + 1
post_employer = employer[employer["rank"] > 1]
post_employer.loc[:,"pre_rank"] = post_employer.loc[:,"rank"] - 1
match_all = employer.merge(post_employer, left_on = ["member_id","rank"], right_on = ["member_id","pre_rank"], how = "left")
match_all[(match_all.company_name_x == "Microsoft") & (match_all.company_name_y == "Google")]["member_id"].nunique()

3